In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [3]:
mask_detection = tf.keras.models.load_model("mask_model_mobileNet.h5", compile=False)

adam_lr = 0.00001
beta1 = 0.99
beta2 = 0.99
eps = 1e-9

optimizer_adam = keras.optimizers.Adam(learning_rate=adam_lr, beta_1=beta1, beta_2=beta2, epsilon=eps)

mask_detection.compile(
    optimizer = optimizer_adam, 
    loss = 'sparse_categorical_crossentropy', 
    metrics = ['accuracy']
)

In [4]:
def predict(image):
    face_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_frame = cv2.resize(face_frame, (224, 224))
    face_frame = img_to_array(face_frame)
    face_frame = np.expand_dims(face_frame, axis=0)
    face_frame =  preprocess_input(face_frame)
    
    # predict the selected face frame for mask
    prediction = mask_detection.predict(face_frame)
    
    return prediction[0][0]

In [9]:
text_mask = "Mask On"
text_no_mask = "Mask Off"
font = cv2.FONT_HERSHEY_SIMPLEX
scale = 0.8

def detect_and_draw(frame):
    # For grayscale image
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Initialize Haar Cascade
    faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_color = frame[y:y+h, x:x+w]

        # Predict the class using your model's "predict" function
        prediction = predict(roi_color)
        #print(prediction)
        # Get the class index with the maximum probability
        class_id = np.argmax(prediction)
        #print(class_id)
        
        if prediction < 1.1e-8:
            label = "WithMask"
            color = (0, 255, 0)  # Red color for WithMask
        elif 1.1e-06 < prediction < 0.9:
            label = "ImproperMask"
            color = (10, 115, 245)  # Yellow color for ImproperMask
        elif 0.1e-2 < prediction < 1.1:
            label = "WithoutMask"
            color = (0, 0, 255)  # Green color for WithoutMask
        else:
            continue


        # Frame size for the rectangular detection frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        # Output text after the predictions made
        cv2.putText(frame, text=label, org=(x+5, y-10), fontFace=font, 
                    fontScale=scale, color=color, thickness=2)


    return frame

In [10]:
# Initialize the video capture
video_cap = cv2.VideoCapture(0)

# Set the desired window size
window_width = 640
window_height = 480
window_name = "Mask Detection Webcam"

cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, window_width, window_height)

while True:
    ret, frame = video_cap.read()
    if not ret:
        break

    # Perform face detection and annotation
    annotated_frame = detect_and_draw(frame)  # Use your detect_and_draw function here

    # Resize and display the frame
    resized_frame = cv2.resize(annotated_frame, (window_width, window_height))
    cv2.imshow(window_name, resized_frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
video_cap.release()
cv2.destroyAllWindows()